# Split single trial timing files

In [22]:
import os
from shutil import rmtree, copyfile

## load timing files

In [23]:
folders = os.listdir(".")

In [24]:
folders = [f for f in folders if all([os.path.isdir(f), f.startswith("19"), len(f) == 5])]

## For trigger timing files, we split each line to a single file

In [25]:
def split_trigger_timing(path, file_name):
    lines = []
    with open(os.path.join(path, file_name), 'r') as f:
        lines = f.readlines()
    file_handle = file_name.rstrip(".txt") 
    for i, line in enumerate(lines):
        with open(os.path.join(path, "single_trial", file_handle + "_" + str(i + 1) + ".txt"), "w") as f:
            f.write(line)

## For strategy timing files, we want to group trials during which the subjects are given the same strategy to one file

In [26]:
def build_strategy_group(file_name):
    """This is for figuring out which trials in strategy to put in the same group
    """
    strategy_dict = {}
    with open(file_name, 'r') as f:
        line_dict = {}
        for i, line in enumerate(f):
            info = line.split("\t")
            info[0] = info[0].strip()
            info[1] = info[1].strip()
            if info[1] in line_dict:
                line_dict[info[1]].append(i % 4)
            else:
                line_dict[info[1]] = [i % 4]
            if i % 4 == 3:
                strategy_dict[i // 4 + 1] = line_dict.values()
                line_dict = {}
    return strategy_dict

In [32]:
def split_strategy_timing(path, file_name, strategy_dict):
    lines = []
    with open(os.path.join(path, file_name), 'r') as f:
        lines = f.readlines()
    file_handle = file_name.rstrip(".txt")
    file_run = file_name.split("_")[1][-1]
    for i, group in enumerate(strategy_dict[int(file_run)]):
        with open(os.path.join(path, "single_trial", file_handle + "_" + str(i + 1) + ".txt"), "w") as f:
            for g in group:
                f.write(lines[g])

## For resting trials, we simply copy them

In [33]:
def copy_rest_timing(path, file_name):
    copyfile(os.path.join(path, file_name), os.path.join(path, "single_trial", file_name))

In [36]:
for folder in folders:
    files = os.listdir(folder)
    sub_path = os.path.join(".", folder, "timing_files", "standard")
    # create a new folder
    single_trial_folder = os.path.join(sub_path, "single_trial")
    if os.path.exists(single_trial_folder):
        rmtree(single_trial_folder)
    os.mkdir(single_trial_folder)
    # create strategy group
    strategy_dict = build_strategy_group(os.path.join(".", folder, folder + "_trial_pairings.txt"))
    # go through folders
    for f in os.listdir(sub_path):
        if f.startswith("CBT") and "train" in f:
            if "TRIGGER" in f:
                split_trigger_timing(sub_path, f)
            elif "STRATEGY" in f:
                split_strategy_timing(sub_path, f, strategy_dict)  
            elif "REST" in f:
                copy_rest_timing(sub_path, f)